In [5]:
!pip install eli5
!pip install shap

     |████████████████████████████████| 358kB 19.2MB/s 
  Created wheel for shap: filename=shap-0.39.0-cp37-cp37m-linux_x86_64.whl size=491629 sha256=fd956f7cb01d8a98e445f1b36f7e12e517e451ac81f190b004b24ec2e35f079a
  Stored in directory: /root/.cache/pip/wheels/15/27/f5/a8ab9da52fd159aae6477b5ede6eaaec69fd130fa0fa59f283
Successfully built shap


In [15]:
### 패키지 설치 
import pandas as pd #Analysis 
import matplotlib.pyplot as plt #Visulization
import seaborn as sns #Visulization
import numpy as np #Analysis 
from scipy.stats import norm #Analysis 
from sklearn.preprocessing import StandardScaler #Analysis 
from scipy import stats #Analysis 
import warnings 
warnings.filterwarnings('ignore')
import gc

import os
import string
color = sns.color_palette()


from plotly import tools
import plotly.offline as py
py.init_notebook_mode(connected=True)
import plotly.graph_objs as go

from sklearn import model_selection, preprocessing, metrics, ensemble, naive_bayes, linear_model
from sklearn.feature_extraction.text import TfidfVectorizer, CountVectorizer
from sklearn.decomposition import TruncatedSVD

pd.options.mode.chained_assignment = None
pd.options.display.max_columns = 999

import plotly.graph_objs as go

import time
import random

from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_absolute_error
from sklearn.model_selection import train_test_split

from lightgbm import LGBMRegressor

### Permutation importance
import eli5
from eli5.sklearn import PermutationImportance


###shap
import shap

# 목차
- 데이터 탐색
  - 결측치 제거 및 유니크 값
  - 시간에 따른 탐색
  - Amount의 심화 탐색
  - installments(할부) 심화 탐색
- 클러스터링
  - 일별 거래의 변화율
  - 시간별 거래의 변화율
  - Prophet을 통한 클러스터 별 트렌드 분석
- 변수중요도
  - Feature Engineering
  - Correlation
  - Tree Importance
  - Permutation Importance
  - Shap value

## 데이터 탐색

In [7]:
file_dir = '/content/drive/MyDrive/kaggle-study/4/data/'

# parse_dates: 바로 datetime 으로 
train = pd.read_csv(file_dir + 'train.csv', parse_dates=['date'])
train.shape

(3362796, 8)

변수는 총 8개이고, 관측치는 330만개로 관측치에 비해 변수가 많이 적은 상황. 그리고 각 변수의 설명은 아래와 같음.

- store_id :각 파일에서의 상점 고유 번호.
- date : 거래 일자
- time :거래 시간
- card_id :카드 번호의 hash 값
- amount :매출액
- installments : 할부개월수. 일시불은 빈 문자열
- days_of_week : 요일, 월요일이 0, 일요일은 6
- holiday : 1이면 공휴일, 0이면 공휴일이 아님

In [8]:
train.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 3362796 entries, 0 to 3362795
Data columns (total 8 columns):
 #   Column        Dtype         
---  ------        -----         
 0   store_id      int64         
 1   date          datetime64[ns]
 2   time          object        
 3   card_id       object        
 4   amount        int64         
 5   installments  float64       
 6   days_of_week  int64         
 7   holyday       int64         
dtypes: datetime64[ns](1), float64(1), int64(4), object(2)
memory usage: 205.2+ MB


In [9]:
train.head()

,store_id,date,time,card_id,amount,installments,days_of_week,holyday
0,0,2016-12-14,18:05:31,d297bba73f,5,NaN,2,0
1,0,2016-12-14,18:05:54,d297bba73f,-5,NaN,2,0
2,0,2016-12-19,12:42:31,0880849c05,144,NaN,0,0
3,0,2016-12-19,12:48:08,8b4f9e0e95,66,NaN,0,0
4,0,2016-12-19,13:31:08,7ad237eed0,24,NaN,0,0


데이터는 크게 3가지로 구분되어 짐. 시간, 식별화 코드, 매출

- 시간 : date, time, days_of_week, holiday
- 식별화 코드 : store_id, card_id
- 매출 : amount, installments
ㅡ> 참고 : 변수명 중에 holyday는 holiday가 오타난것 같음

아이디어

- row 1을 보면 amount가 음수인게 있는데 이는 취소된 거래를 말하는 것임. 바로 위 row 0의 거래가 3초만에 취소되었다는것을 의미.
- date와 time의 경우 좀 더 구분의 편의를 위해서 년도/월/일/시간/분으로 변수를 확장 함.
- days_of_week도 월~금, 토 ~ 일요일인지 여부를 나누어서 비교.
- installments에 보면 결측치가 있는데, 이는 일시불을 의미하는것이므로 1으로 처리.

## 결측치 제거

In [16]:
#https://www.kaggle.com/ashishpatel26/bird-eye-view-of-two-sigma-nn-approach
def mis_value_graph(data):  
    data = [
    go.Bar(
        x = data.columns,
        y = data.isnull().sum(),
        name = 'Counts of Missing value',
        textfont=dict(size=20),
        marker=dict(
        line=dict(
            color= generate_color(),
            #width= 2,
        ), opacity = 0.45
    )
    ),
    ]
    layout= go.Layout(
        title= '"Total Missing Value By Column"',
        xaxis= dict(title='Columns', ticklen=5, zeroline=False, gridwidth=2),
        yaxis= dict(title='Value Count', ticklen=5, gridwidth=2),
        showlegend=True
    )
    fig = go.Figure(data=data, layout=layout)
    py.iplot(fig, filename='skin')
    
def generate_color():
    color = '#{:02x}{:02x}{:02x}'.format(*map(lambda x: random.randint(0, 255), range(3)))
    return color

mis_value_graph(train)

In [14]:

### 결측치 처리 코드 
train['installments'] = train['installments'].fillna(1)

In [17]:
### 유니크 갯수 계산
train_unique = []
for i in train.columns.values:
    train_unique.append(len(train[i].unique()))
unique_train = pd.DataFrame()
unique_train['Columns'] = list(train.columns)
unique_train['Unique_value'] = train_unique

In [18]:
data = [
    go.Bar(
        x = unique_train['Columns'],
        y = unique_train['Unique_value'],
        name = 'Unique value in features',
        textfont=dict(size=20),
        marker=dict(
        line=dict(
            color= generate_color(),
            #width= 2,
        ), opacity = 0.45
    )
    ),
    ]
layout= go.Layout(
        title= "Unique Value By Column",
        xaxis= dict(title='Columns', ticklen=5, zeroline=False, gridwidth=2),
        yaxis= dict(title='Value Count', ticklen=5, gridwidth=2),
        showlegend=True
    )
fig = go.Figure(data=data, layout=layout)
py.iplot(fig, filename='skin')

In [19]:
train['installments'].unique()

array([ 1.,  3.,  2.,  4.,  5., 10.,  6., 12.,  7.,  8.,  9., 22., 24.,
       18., 20., 15., 36.])

In [20]:
###요일변수
train['date'] = pd.to_datetime(train['date'])
train['year'] = train['date'].dt.year
train['month'] = train['date'].dt.month
train['day'] = train['date'].dt.day

train['hour'] = train['time'].apply(lambda x: x[0:2])
train['hour'] = train['hour'].astype('int')

train['weekend'] = train['days_of_week'].apply(lambda x: 1 if x < 4 else 0)

In [21]:
### 일별 Store_id의 구매건수
def scatter_plot(cnt_srs, color, showlegend1=False, name1 = ""):
    trace = go.Scatter(
        x=cnt_srs.index[::-1],
        y=cnt_srs.values[::-1],
        name = name1,
        showlegend=showlegend1,
        marker=dict(
            color=color,
        ),
    )
    return trace

train['date'] = pd.to_datetime(train['date'])

cnt_srs = train.groupby('date')['store_id'].size()


trace = scatter_plot(cnt_srs, 'red')

layout = go.Layout(
    height=600,
    width=1000,
    #paper_bgcolor='rgb(233,233,233)',
    title='Dates in train set'
)

data = [trace]
fig = go.Figure(data=data, layout=layout)
py.iplot(fig, filename="ActivationDate")